In [1]:
import numpy as np
import gurobipy as gb
import pylab as pl

from src import *

# TSDAP
## Time Dependent Activity Scheduling Problem

In [2]:
Q = 200
n_activities = 100
eps = 0.5
discretizer = TimeDiscretizer(eps)
activities = [Activity(i, 4*(i+1), my_tau, my_rho) for i in range(n_activities)]

V = []
q = []
for activity in activities:
    times = discretizer.discretize_activity(activity)
    V.append(dict([(i, time) for i, time in enumerate(times)]))
    q.append(dict([(i, activity.resources(time)) for i, time in enumerate(times)]))

In [17]:
final_activity, l, p = TEN_solve(V, activities, q, Q)

Solved!


In [18]:
activities[-1].completion_time(V[-1][final_activity[1]])

257.60403574705543

In [19]:
for t, time in p[-1].items():
    if time is not None:
        last = time
        break

path = [final_activity, last]
while last[0] > 0:
    last = p[last[0]][last[1]]
    path.append(last)

path[::-1]

[(0, 2),
 (1, 6),
 (2, 7),
 (3, 13),
 (4, 15),
 (5, 17),
 (6, 21),
 (7, 22),
 (8, 28),
 (9, 30),
 (10, 32),
 (11, 36),
 (12, 37),
 (13, 39),
 (14, 43),
 (15, 44),
 (16, 50),
 (17, 52),
 (18, 53),
 (19, 56),
 (20, 59),
 (21, 65),
 (22, 67),
 (23, 69),
 (24, 73),
 (25, 74),
 (26, 80),
 (27, 82),
 (28, 84),
 (29, 88),
 (30, 89),
 (31, 91),
 (32, 95),
 (33, 96),
 (34, 102),
 (35, 104),
 (36, 106),
 (37, 110),
 (38, 111),
 (39, 117),
 (40, 119),
 (41, 121),
 (42, 125),
 (43, 126),
 (44, 132),
 (45, 134),
 (46, 136),
 (47, 140),
 (48, 141),
 (49, 146),
 (50, 149),
 (51, 156),
 (52, 158),
 (53, 164),
 (54, 167),
 (55, 173),
 (56, 175),
 (57, 177),
 (58, 181),
 (59, 182),
 (60, 188),
 (61, 190),
 (62, 192),
 (63, 196),
 (64, 197),
 (65, 202),
 (66, 205),
 (67, 207),
 (68, 211),
 (69, 212),
 (70, 218),
 (71, 221),
 (72, 227),
 (73, 229),
 (74, 230),
 (75, 233),
 (76, 236),
 (77, 242),
 (78, 244),
 (79, 246),
 (80, 250),
 (81, 251),
 (82, 257),
 (83, 259),
 (84, 261),
 (85, 265),
 (86, 266),
 (8

In [85]:
tdasp = gb.Model()
x = tdasp.addMVar((n_activities, n_steps), vtype=gb.GRB.BINARY)

In [3]:
assignment = gb.Model()
assignment.modelSense = gb.GRB.MINIMIZE #declare mimization

# assignment.setParam('OutputFlag', 0) suppress outputs, equivalent of xpress setControl('outputlog', 0)

X = assignment.addVars( [(i,j) for i in range(4) for j in range(4)], vtype=gb.GRB.BINARY) #this way of declare vars does not allow to work with matrix multiplication (X is a tuple dict)

I = range(4)
J = range(4)

costs = np.array([[13,	4,	7, 6],
                	[1,	11,	5, 4],
                  [6,	7,	2, 8],
                  [1,	3,	5, 9]])


for i in I:
  assignment.addConstr( gb.quicksum(X[i,j] for j in J) <= 1) #quicksum is the equivalent to xp.Sum

for j in J:
  assignment.addConstr( gb.quicksum(X[i,j] for i in I) == 1 )

assignment.setObjective( 
    gb.quicksum( costs[i,j]*X[i,j]   for j in J for i in I)
    )   

assignment.optimize() #equivalent to solve() for xpress


Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x6f06b897
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 1.100000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      11.0000000   11.00000  0.00%     -    0s

Explored 1 nodes (